# Train neural networks with `momics` and `tensorflow`

`momics` provides several useful resources to train neural networks with `tensorflow`. This notebook demonstrates how to train a simple neural network with `momics` and `tensorflow`.

## Connect to the data repository

We will tap into the repository generated in the [previous tutorial](integrating-multiomics). 

In [ ]:
from momics.momics import Momics

## Creating repository
repo = Momics("yeast_CNN_data.momics")

## Check that sequence and some tracks are registered
repo.seq()
repo.tracks()

## Modify some tracks

We can first pre-process the tracks to normalize them, and save them back to the local repository.

In [ ]:
import numpy as np

for track in ["ATAC", "MNase"]:
    cov = repo.tracks(track)
    q99 = np.nanpercentile(np.concatenate(list(cov.values())), 99)
    for chrom in cov.keys():
        arr = cov[chrom]
        arr = np.minimum(arr, q99)
        arr = (arr - np.nanmin(arr)) / (np.nanmax(arr) - np.nanmin(arr))
        cov[chrom] = np.nan_to_num(arr, nan=0)
    repo.ingest_track(cov, track + "_rescaled")

repo.tracks()

## Define datasets and model 

We will define a simple convolutional neural network with `tensorflow` to predict the target variable `ATAC` from the feature variable `MNase`. This requires to first define a set of genomic coordinates to extract genomic data from. We will use `MNase_rescaled` coverage scores over tiling genomic windows (`features_size` of `1025`, with a stride of `48`) as feature variables to predict `ATAC_rescaled` coverage scores over the same tiling genomic windows, but narrowed down to the a `target_size` of `24` bp around the center of the window. We can split the data into training, testing and validation sets, using `momics.utils.split_ranges()`.

In [ ]:
import momics.utils as mutils

# Fetch data from the momics repository
features = "MNase_rescaled"
target = "ATAC_rescaled"
features_size = 1024 + 1
target_size = 24
stride = 48

bins = repo.bins(width=features_size, stride=stride, cut_last_bin_out=True)
bins = bins.subset(lambda x: x.Chromosome != "XVI")
bins_split, bins_test = mutils.split_ranges(bins, 0.8, shuffle=True)
bins_train, bins_val = mutils.split_ranges(bins_split, 0.8, shuffle=True)

We now need to define different datasets, for training, testing and validation. We will use `momics.dataset.MomicsDataset()` constructor, indicating the batch size we wish to use in the training process.

In [ ]:
from momics.dataset import MomicsDataset

batch_size = 1000

train_dataset = (
    MomicsDataset(repo, bins_train, features, target, target_size=target_size, batch_size=batch_size)
    .shuffle(10)
    .prefetch(2)
    .repeat()
)
val_dataset = MomicsDataset(repo, bins_val, features, target, target_size=target_size, batch_size=batch_size).repeat()
test_dataset = MomicsDataset(repo, bins_test, features, target, target_size=target_size, batch_size=batch_size)
print(train_dataset)

Now is time to define the model architecture. In this example, we will use a simple convolutional neural network (`ChromNN`), pre-defined in `momics.chromnn`. We can instantiate the model, and compile it with the desired optimizer, loss function and metrics.

In [7]:
from momics.chromnn import ChromNN
import tensorflow as tf
from tensorflow.keras import layers  # type: ignore

model = ChromNN(
    layers.Input(shape=(features_size, 1)),
    layers.Dense(target_size, activation="linear"),
).model

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss="mse")
model.summary()

## Fit the model 

Now that we have the datasets and the model, we can fit the model to the training data, using the `fit()` method of the model. We can also evaluate the model on the testing and validation datasets.

In [ ]:
from pathlib import Path
from tensorflow.keras.callbacks import CSVLogger, EarlyStopping, ModelCheckpoint, ReduceLROnPlateau  # type: ignore

callbacks_list = [
    CSVLogger(Path(".chromnn", "epoch_data.csv")),
    ModelCheckpoint(filepath=Path(".chromnn", "Checkpoint.keras").name, monitor="val_correlate", save_best_only=True),
    EarlyStopping(monitor="val_loss", patience=10, min_delta=1e-5, restore_best_weights=True),
    ReduceLROnPlateau(monitor="val_loss", factor=0.1, patience=6 // 2, min_lr=0.1 * 0.001),
]
model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=30,
    callbacks=callbacks_list,
    steps_per_epoch=len(bins_train) // batch_size,
    validation_steps=len(bins_val) // batch_size,
)

## Evaluate and save model 

Now let's see how the trained model performs, and save it to the local repository.

In [5]:
# Evaluate the model
model.evaluate(test_dataset)

# Save the model
model.save("chromnn_model.keras")

## Use the model to predict ATAC-seq coverage

We can now use our trained model to predict ATAC-seq coverage from MNase-seq coverage, for example on a chromosome which has not been used for training.

In [ ]:
from momics.query import MomicsQuery
from momics.aggregate import aggregate

## Define tiling 1025 bp windows, with a stride of 8 bp, and extract MNase data from it.
bb = repo.bins(width=features_size, stride=8, cut_last_bin_out=True)["XVI"]
dat = MomicsQuery(repo, bb).query_tracks(tracks=["MNase_rescaled"])
dat = np.array(list(dat.coverage["MNase_rescaled"].values()))

## Now predict the ATAC signal from the MNase signal
predictions = model.predict(dat)

## Export predictions as a bigwig
bb2 = bb.copy()
bb2.Start = bb2.Start + features_size // 2 - target_size // 2
bb2.End = bb2.Start + target_size
chrom_sizes = {chrom: length for chrom, length in zip(repo.chroms().chrom, repo.chroms().length)}
keys = [f"{chrom}:{start}-{end}" for chrom, start, end in zip(bb2.Chromosome, bb2.Start, bb2.End)]
res = {"atac": {k: None for k in keys}}
for i, key in enumerate(keys):
    res["atac"][key] = predictions[i]

aggregate(res, bb2, chrom_sizes, type="mean", prefix="prediction")

This generates a new `bw` file with ATAC-seq coverage over chr16, predicted from MNase-seq coverage.

## Use other feature variables for prediction

We can use other predictor variable, for example the `nucleotide` sequence of tiling genomic windows, to predict `ATAC_rescaled` coverage scores. We can use the same procedure as above, but with a different set of genomic coordinates and feature variables.

In [ ]:
import numpy as np
import momics.utils as mutils

for track in ["ATAC", "MNase"]:
    cov = repo.tracks(track)
    q99 = np.nanpercentile(np.concatenate(list(cov.values())), 99)
    for chrom in cov.keys():
        arr = cov[chrom]
        arr = np.minimum(arr, q99)
        arr = (arr - np.nanmin(arr)) / (np.nanmax(arr) - np.nanmin(arr))
        cov[chrom] = np.nan_to_num(arr, nan=0)
    repo.ingest_track(cov, track + "_rescaled")

repo.tracks()


# Fetch data from the momics repository
features = "MNase_rescaled"
target = "ATAC_rescaled"
features_size = 1024 + 1
target_size = 24
stride = 48

bins = repo.bins(width=features_size, stride=stride, cut_last_bin_out=True)
bins = bins.subset(lambda x: x.Chromosome != "XVI")
bins_split, bins_test = mutils.split_ranges(bins, 0.8, shuffle=True)
bins_train, bins_val = mutils.split_ranges(bins_split, 0.8, shuffle=True)